In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
DATA_DIR = './data/'
CANCER_DATA = 'cancer_incd_rate_2016_2020.csv'
AQI_DATA = lambda year: f'annual_aqi_by_county_{year}.csv'

In [4]:
def get_cancer_data():
    df = pd.read_csv(DATA_DIR + CANCER_DATA, skiprows=8, skipfooter=31)
    df.insert(0, 'State', df['County'].apply(lambda x: x.split(', ')[-1][:-3]))
    df['County'] = df['County'].apply(lambda x: x.split(', ')[0])
    return df.iloc[1:]
get_cancer_data()

C:\Users\WOODBURNKB20\AppData\Local\Temp\ipykernel_13592\587057053.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(DATA_DIR + CANCER_DATA, skiprows=8, skipfooter=31)


,State,County,FIPS,"Age-Adjusted Incidence Rate([rate note]) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,CI*Rank([rank note]),Lower CI (CI*Rank),Upper CI (CI*Rank),Average Annual Count,Recent Trend,Recent 5-Year Trend ([trend note]) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1
1,Florida,Union County,12125,1237.4,1165.6,1312.8,N/A,1,1,237,stable,0.6,-0.5,1.9
2,Iowa,Palo Alto County,19147,658.1,591.1,731.1,N/A,1,6,82,rising,4.8,0.2,15.4
3,Montana,Treasure County,30103,652.2,401,1007.4,N/A,1,55,7,stable,-1.1,-5.6,3.3
4,Texas,Polk County,48373,633.6,604.6,663.7,N/A,1,4,425,rising,2.2,1.2,4.2
5,Kentucky,Floyd County,21071,616.8,584.3,650.7,N/A,1,19,295,stable,1.5,-1.8,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,Kansas,Wyandotte County,20209,data not available,data not available,data not available,N/A,N/A,N/A,data not available,data not available,data not available,data not available,data not available
3140,Virginia,Wythe County,51197,data not available,data not available,data not available,N/A,N/A,N/A,data not available,data not available,data not available,data not available,data not available
3141,Alaska,Yakutat City and Borough,2282,*,*,*,*,*,*,3 or fewer,*,*,*,*
3142,Minnesota,Yellow Medicine County,27173,data not available,data not available,data not available,N/A,N/A,N/A,data not available,data not available,data not available,data not available,data not available


In [7]:
# compile aqi data into 5 year avaerage
def compile_aqi_data():
    dfs = []
    
    for year in range(2016, 2021):
        aqi = pd.read_csv(DATA_DIR + AQI_DATA(year))
        dfs.append(aqi)
    
    annual_aqi_2016_2020 = pd.concat(dfs)
    
    return annual_aqi_2016_2020

compile_aqi_data()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,Alabama,Baldwin,2016,279,247,32,0,0,0,0,87,51,37,0,0,221,58,0
1,Alabama,Clay,2016,116,109,7,0,0,0,0,56,45,30,0,0,0,116,0
2,Alabama,Colbert,2016,282,258,23,1,0,0,0,115,50,38,0,0,219,63,0
3,Alabama,DeKalb,2016,348,304,43,1,0,0,0,119,54,40,0,0,321,27,0
4,Alabama,Elmore,2016,117,107,10,0,0,0,0,77,48,40,0,0,117,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,Wyoming,Sublette,2020,366,297,64,4,1,0,0,151,61,44,0,0,355,11,0
999,Wyoming,Sweetwater,2020,366,238,116,7,4,0,1,1250,76,46,0,5,253,7,101
1000,Wyoming,Teton,2020,366,318,36,7,5,0,0,161,54,42,0,0,329,37,0
1001,Wyoming,Uinta,2020,366,323,41,2,0,0,0,122,51,40,0,2,225,0,139
